In [ ]:
%load_ext autoreload
%autoreload 2
from youtube_transcript_api import YouTubeTranscriptApi
from yt_dlp import YoutubeDL
import os
from thumbnail_generator import extract_frames, video_id, PromptGenerator, Diffuser

url = "https://www.youtube.com/watch?v=sw5stkAyP60"
yt_str = "https://www.youtube.com/watch?v="
ext = "webm"
res = "480"
gen_res = (1344,768)

id = video_id(url)
transcript_list = YouTubeTranscriptApi.get_transcript(id)
transcript = ' '.join([section['text'] for section in transcript_list])

opts = {
  'format': f'bestvideo[ext={ext}][height={res}]',
  'geo_bypass': True,
  'geo_bypass_country': 'US',
  'paths': {'home': ''},
  'outtmpl': '%(id)s.%(ext)s',
  # 'ignoreerrors': True,
  'quiet': True,
}
with YoutubeDL(opts) as ydl: ydl.download(yt_str + id)
frames = extract_frames(f"{id}.{ext}", frame_amt=1)
answer = PromptGenerator.moondream(transcript, f"{id}.{ext}")
os.remove(f"{id}.{ext}")
diff = Diffuser()
lora = "sdxl/1344x768-200-1600-500-cats-no"
diff.pipe.load_lora_weights(f"loras/{lora}.safetensors")
diff.generate(answer["positive"], batch_size=4, width=gen_res[0], height=gen_res[1], seed=42)
display(diff.get_grid())

In [ ]:
print(answer)

In [ ]:
from thumbnail_generator import Youtube, PromptGenerator

yt = Youtube("dataset/first")
videos = yt.videos
v = videos[1]
id = v["id"]
transcript = ' '.join([v["text"] for v in v["caption"]])
print(transcript)
print(id)

In [ ]:
# using the moondream model
# PromptGenerator.moondream(transcript, vid_path = "a01c.avi")
# using the moondream model, finetuned on ...
res = PromptGenerator.moondream(transcript, vid_path = f"videos/{id}.webm", ft_path = "moondream_ft_moon_mean_eps10_bs8")
print(res)